In [115]:
import csv
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [116]:
data = pd.read_csv('C:\\Users\\Eswar\\Downloads\\all_tickets.csv')

#drop rows which has category as 0
data = data[data.category != 0]

#drop rows which has particular value
data = data[data['body'].notna()]

# taking
df = data[["body","category"]]

df.head()



,body,category
0,hi since recruiter lead permission approve req...,4
1,icon dear please setup icon per icon engineers...,6
2,work experience user hi work experience studen...,5
3,requesting meeting hi please help follow equip...,5
4,re expire days hi ask help update passwords co...,4


In [117]:
# checking the lables quantity
lable_count = df['category'].value_counts()
print(df.shape)
lable_count

(48545, 2)


4     34061
5      9634
6      2628
7       921
11      612
8       239
9       191
3       137
1        72
12       45
2         3
10        2
Name: category, dtype: int64

In [118]:
# removing less featured category
category_removal = [12,10,8,9,3,1,2]

for col in category_removal:
    df = df[df.category != col]

In [119]:
# checking the lables quantity

lable_count = df['category'].value_counts()
print(df.shape)
lable_count

(47856, 2)


4     34061
5      9634
6      2628
7       921
11      612
Name: category, dtype: int64

In [120]:
# Balencing features

main_dataframe_list = []
for category_numbers in lable_count.keys():
    data_frame = df.loc[df['category'] == category_numbers].iloc[0:500]
    main_dataframe_list.append(data_frame)

df = pd.concat(main_dataframe_list)

In [121]:
df['category'].value_counts()

4     500
5     500
6     500
7     500
11    500
Name: category, dtype: int64

In [122]:
df.head()

,body,category
0,hi since recruiter lead permission approve req...,4
4,re expire days hi ask help update passwords co...,4
5,verification warning hi has got attached pleas...,4
6,please dear looks blacklisted receiving mails ...,4
8,dear modules report report cost thank much reg...,4


In [123]:
# Shuffling and sorting index

df = df.sample(frac=1).reset_index(drop=True)

In [124]:
df.head()

,body,category
0,install dear please log assign helping her ins...,5
1,purchase po dear purchased link subscription a...,5
2,tuesday february pm card si ca card,7
3,board hello please add individuals black regards,4
4,report user hi seems crack take appropriate ac...,5


In [125]:
df.tail()

,body,category
2495,re card hello va si,7
2496,access card issue hi have issue with card can ...,11
2497,allocation hello please allocate assigned temp...,5
2498,sent monday july badge working hello there see...,11
2499,disk problem hi more disk partition please pro...,6


#talking the values to CSV - for observation

sample = pd.concat([df.iloc[[0]]
,df.iloc[[1]]
,df.iloc[[2]]
,df.iloc[[3]]
,df.iloc[[2495]]])

sample.to_csv('observe.csv')



In [126]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import string
import re
port_stemmer = nltk.PorterStemmer()


In [127]:
def cleaning(text):

    # punctuation removing
    Punctuation_rm = re.sub("[^-9A-Za-z ]", "" , text)

    # creating tokens
    tokens =  nltk.tokenize.word_tokenize(Punctuation_rm)

    # Stopword removing
    rem_text = [i for i in tokens if i not in stop_words]

    stem = " ".join([port_stemmer.stem(word) for word in rem_text])


    return stem

# remove stopwords from the text
df['cleaned_body']= df['body'].apply(cleaning)

In [128]:


print(df.shape)
df.tail()

(2500, 3)


,body,category,cleaned_body
2495,re card hello va si,7,card hello va si
2496,access card issue hi have issue with card can ...,11,access card issu hi issu card access locat day...
2497,allocation hello please allocate assigned temp...,5,alloc hello pleas alloc assign temporarili per...
2498,sent monday july badge working hello there see...,11,sent monday juli badg work hello seem kind pro...
2499,disk problem hi more disk partition please pro...,6,disk problem hi disk partit pleas provid least...


In [129]:
#  Spliting data [ Testing , training ]

X_train = df.loc[:1500, 'cleaned_body'].values
y_train = df.loc[:1500, 'category'].values
X_test = df.loc[1501:, 'cleaned_body'].values
y_test = df.loc[1501:, 'category'].values

In [130]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)
print(train_vectors.shape, test_vectors.shape)

(1501, 2504) (999, 2504)


In [131]:
from sklearn.naive_bayes import MultinomialNB
from  sklearn.metrics  import accuracy_score
model = MultinomialNB().fit(train_vectors, y_train)

In [132]:

predicted = model.predict(test_vectors)
print(accuracy_score(y_test,predicted))

0.7447447447447447


In [133]:
test = ['tuesday march pm visitor cards review hi please each location where visitor cards make information control matches reality physical cards oms possession discrepancy must corrected appropriately recorded spreadsheet liaise oms each location questions please hesitate thanks']

s = vectorizer.transform(test)

predicted = model.predict(s)

In [134]:
predicted

array([7], dtype=int64)

In [136]:
# loading library
import pickle


# create an iterator object with write permission - model.pkl
with open('model_pkl', 'wb') as files:
    pickle.dump(model, files)

filename = "model_pk"

pickle.dump(model,open(filename,"wb"))

In [137]:
# load saved model

loadedModel = pickle.load(open(filename,"rb"))


In [143]:
s = "tuesday march pm visitor cards review hi please each location where visitor cards make information control matches reality physical cards oms possession discrepancy must corrected appropriately recorded spreadsheet liaise oms each location questions please hesitate thanks"

teststr = vectorizer.transform([s])
loadedModel.predict(teststr)

array([7], dtype=int64)